# Day 21

## Part I

本日是复杂的集合运算，基本就是二维表中的基本运算实现。对于使用Python来说，无疑Pandas是最合适的工具。我们首先需要将输入数据处理成一个Pandas的DataFrame，其行index是所有输入中出现的那些乱七八糟的字符串（原料名称），其列columns是所有输入中出现的那些过敏反应。整张二维表默认初始化为全0，表示暂无配方和过敏源，然后再次遍历每条配方，每次出现配方的原料和相应的过敏反应时，都在原表的基础上加一，最后输出这个DataFrame和配方列表。下面是读取输入函数的定义：

In [1]:
from typing import List, Tuple
import pandas as pd
import numpy as np

def read_input(input_file: str) -> Tuple[pd.DataFrame, List[List[str]]]:
    # 用来记录所有的过敏反应和原料
    allergens = set()
    ingredients = set()
    # 所有的配方，最后作为元组的第二个返回值
    formulas = []
    with open(input_file) as fn:
        lines = fn.readlines()
    # 第一次遍历，将所有过敏反应和原料都放在集合中，最后用来创建一个全0的DataFrame
    for line in lines:
        part1, part2 = line.rstrip().split(' (')
        for ing in part1.split(' '):
            ingredients.add(ing)
        for aller in part2[9:].rstrip(')').split(', '):
            allergens.add(aller)
    df = pd.DataFrame(np.zeros((len(ingredients), len(allergens)), dtype=int), 
                      index=ingredients, columns=allergens)
    
    # 第二次遍历，将每条配方中原料和过敏反应都累加，并产生配方列表
    for line in lines:
        part1, part2 = line.rstrip().split(' (')
        formulas.append(part1.split(' '))
        for aller in part2[9:].rstrip(')').split(', '):
            df.loc[part1.split(' '), aller] += 1
    return df, formulas

下面是关键逻辑函数，在整个DataFrame中找到所有对应的过敏源。这个函数的逻辑是，用一个字典记录所有已经找到的过敏源对应关系，直到找到所有的过敏反应为止。如果DataFrame中有一列只有一个最大值，那么这个最大值对应的index就是这个过敏反应的过敏源，将其加入字典中后，再将这个单元格（过敏反应和原料对应）所在的行和列都全部重置为0，表示这个原料已经不可能是其他过敏反应的源，同样这个过敏反应也不再可能是由其他原料引起的（题设中的1对1关系）。当所有的过敏源都找到后，DataFrame将会全部重置为0：

In [2]:
def find_all_source(df: pd.DataFrame) -> Tuple[List[str], List[str]]:

    known_allergens = {}

    while len(known_allergens) != len(df.columns):
        for col in df:
            max_rows = df[df[col]==df[col].max()]
            if len(max_rows) == 1:
                index = max_rows.index[0]
                known_allergens[col] = index
                df.loc[index] = 0
                df[col] = 0
    
    # 将找到的过敏源对应关系，按照过敏反应排序，然后输出到一个列表中，这是第二部分的问题
    aller_ings = [v[1] for v in sorted(known_allergens.items(), key=lambda x: x[0])]

    return list(set(df.index).difference(known_allergens.values())), aller_ings

## Part II

本来下面应该是第一部分的逻辑，但是完成后发现第二部分其实也一并解决了，于是就将两个部分的实现逻辑合并成了下面一个函数：

In [3]:
def all_solution(df: pd.DataFrame, formulas: List[List[str]]) -> Tuple[int, str]:
    sources, aller_ings = find_all_source(df)
    counter = 0
    # 第一部分要计算不是过敏源的原料在所有配方中出现的次数总和
    for source in sources:
        for formula in formulas:
            if source in formula:
                counter += 1
    # 返回第一部分结果和第二部分结果
    return counter, ','.join(aller_ings)

单元测试：

In [4]:
test_df, test_formulas = read_input('testcase1.txt')
assert(all_solution(test_df, test_formulas) == (5, 'mxmxvkd,sqjhc,fvjkl'))

两个部分结果一起算出来：

In [5]:
df, formulas = read_input('input.txt')
all_solution(df, formulas)

(2287, 'fntg,gtqfrp,xlvrggj,rlsr,xpbxbv,jtjtrd,fvjkp,zhszc')